# Restaurant Visitor Forecasting


Restaurants need to know the number of visitors each day to plan for supplies and personnel. This is however a difficult task due to many unpredictable factors. Recruit Holdings has brought data related to Restaurants, Restaurant Reservations & Restaurant Visits from the following sites, which will be our input data for analysis:

*    Hot Pepper Gourmet: A Restaurant-rating service similar to Yelp!
*    AirREGI:  Reservation control and cash register used at Restaurants to make transactions




## 1. Goal of Competition
Predict total number of visitors to a restaurant for specified future dates

## 2. Dataset Summary

*   Data is orignated from 2 systems - prefaced with air or hpg
*   Restaurant has unique Id - either a air_store_id or hpg_store_id
*   Not all restaurants are covered by both systems
*   Restaurant data more than what is required for prediction is provided
*   Latitude & Longitude are not exact
*   Training Dataset: Data from January 2016 to early (first week) April 2017
*   Test Dataset: Predict Visitors for mid weeks (second and third weeks) of April 2017
*   The training and testing set both omit days where the restaurants were closed.       


## 3. Setup Environment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns

from datetime import datetime
from sklearn import preprocessing

## 4. Load Data

In [ ]:
hpg_store_info = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/MetaData/MetaData/hpg_store_info.csv")
hpg_reserve = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/MetaData/MetaData/hpg_reserve.csv")
air_store_info = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/MetaData/MetaData/air_store_info.csv")
air_reserve = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/MetaData/MetaData/air_reserve.csv")
store_id_relation = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/MetaData/MetaData/store_id_relation.csv")
date_info = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/MetaData/MetaData/date_info.csv")

test = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/sample_submission.csv")
test_ = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/test.csv")
train = pd.read_csv("/kaggle/input/restaurant-visitor-forecasting/train.csv")

## 5. Data Files & Description

*    air_store_info.csv: Information about stores in air system
*    hpg_store_info.csv: Information about stores in hpg system
*    store_id_relation.csv: Mapping between stores in air and hpg system
*    air_reserve.csv: Reservations made in air system
*    hpg_reserve.csv: Reservations made in hpg system
*    date_info.csv: Holidays in Japan for the period relevant to dataset
*    train.csv: Historical visit data for air restaurants
*    test.csv: Future restaurant visit date for which number of visitors are required to be predicted 

### 5.1. Basic Overview:

In [ ]:
print("Total restaurants common in AIR and HPG: ",len(store_id_relation))

print("\n\nAIR dataset overview")
print("Total number of entries in air_reserve dataset: ", air_reserve.shape)
print("Total number of restaurants in air_reserve dataset: ", len(air_reserve.air_store_id.unique()))
print("Total unique genre in AIR restaurants: ",len(air_store_info.air_genre_name.unique()))
print("Total number of AIR restaurant's locations: ",len(air_store_info.air_area_name.unique()))

print("\n\nHPG dataset overview")
print("Total number of entries in hpg_reserve dataset: ", hpg_reserve.shape)
print("Total number of restaurants in hpg_reserve dataset: ", len(hpg_reserve.hpg_store_id.unique()))
print("Total unique genre in HPG restaurants: ",len(hpg_store_info.hpg_genre_name.unique()))
print("Total number of HPG restaurant's locations: ",len(hpg_store_info.hpg_area_name.unique()))

print("\n\nTraining dataset overview")
print("Total number of entries in train dataset: ", train.shape)
print("Total number of unique AIR restaurants in the train set: ",len(train.air_store_id.unique()))
print("Average daily visitors: ",train.visitors.mean())
print("Training data duration:- ", train.visit_date.min(), " to " ,train.visit_date.max())

print("\n\nTest dataset overview")
print("Total number of entries in test dataset: ", test_.shape)
print("Total unique restaurants:-",len(test_.air_store_id.unique()))
print("Test data duration: ", test_.visit_date.min(), " to ",test_.visit_date.max())

### 5.2. Null Value Analysis : 

In [ ]:
print("AIR_STORE_INFO:")
print(air_store_info.isnull().sum())

print("\nHPG_STORE_INFO:")
print(hpg_store_info.isnull().sum())

print("\nAIR_RESERVE:")
print(air_reserve.isnull().sum())

print("\nHPG_RESERVE:")
print(hpg_reserve.isnull().sum())

print("\nSTORE_ID_RELATION:")
print(store_id_relation.isnull().sum())


print("\nDATE_INFO:")
print(date_info.isnull().sum())


print("\nTRAIN:")
print(train.isnull().sum())

print("\nTEST:")
print(test.isnull().sum())

**Observations:-**

*    From the above outputs we can observe that there are **no null values** in the dataset.
*    There is no need of performing any kind of missing data imputation.



### 5.3. Preprocessing the data:

In [ ]:
# preprocessing air visitors from TRAIN dataset

air_data = pd.merge(train,air_store_info,how='left', on=['air_store_id']) # merging dataframes
date_info.rename(columns={'calendar_date':'visit_date'},inplace=True)  # renaming columns
air_data = pd.merge(air_data,date_info,how='left', on=['visit_date'])
air_data.sort_values(by='visit_date',ignore_index=True,inplace=True)

air_data['visit_date'] = pd.to_datetime(air_data['visit_date'])
# air_data['day'] = air_data['visit_date'].dt.day
# air_data['dow'] = air_data['visit_date'].dt.weekday
# air_data['year'] = air_data['visit_date'].dt.year
# air_data['month'] = air_data['visit_date'].dt.month
air_data['month_name'] = air_data['visit_date'].dt.month_name()
# air_data['week'] = air_data['visit_date'].dt.isocalendar().week
# air_data['quarter'] = air_data['visit_date'].dt.quarter
air_data['visit_date'] = air_data['visit_date'].dt.date
# air_data.to_csv('air_data.csv',index=False)

In [ ]:
air_data.head()

*   We are merging the related datasets with the training data  in  the **air_data** dataframe to get all the features for further comparisions with the number of visitors.
*   All the date-related informations are extracted from the visit_date column and added to air_data dataframe as new features. 

In [ ]:
# preprocessing AIR reservation data
air_reserve_data = pd.merge(air_reserve,air_store_info,how='left', on=['air_store_id'])

air_reserve_data['visit_datetime'] = pd.to_datetime(air_reserve_data['visit_datetime'])
air_reserve_data['visit_hour'] = air_reserve_data['visit_datetime'].dt.hour
air_reserve_data['visit_date'] = air_reserve_data['visit_datetime'].dt.date

air_reserve_data['reserve_datetime'] = pd.to_datetime(air_reserve_data['reserve_datetime'])
air_reserve_data['reserve_hour'] = air_reserve_data['reserve_datetime'].dt.hour
air_reserve_data['reserve_date'] = air_reserve_data['reserve_datetime'].dt.date

#calculate reservation time difference 
air_reserve_data['res_vis_diff'] = (air_reserve_data['visit_datetime']-air_reserve_data['reserve_datetime']).apply(
    lambda x : x.total_seconds()/3600)

air_reserve_data.rename(columns={'reserve_visitors':'air_reserve_visitors'},inplace=True)
air_reserve_data.to_csv('air_reserve_data.csv',index=False)


In [ ]:
air_reserve_data.head()

*   The air_reserve data is merged with air_store_info to get the area ,genre, latitude and longitude for the corresponding air_store_id into **air_reserve_data** dataframe.
*   The visit_datetime column has been changed into datetime from object dtype.

In [ ]:
# preprocessing HPG reservation data
hpg_reserve_data = pd.merge(hpg_reserve,store_id_relation,on=['hpg_store_id'],how='inner')
hpg_reserve_data = pd.merge(hpg_reserve_data,hpg_store_info,on=['hpg_store_id'],how='left')

hpg_reserve_data['visit_datetime'] = pd.to_datetime(hpg_reserve_data['visit_datetime'])
hpg_reserve_data['visit_hour'] = hpg_reserve_data['visit_datetime'].dt.hour
hpg_reserve_data['visit_date'] = hpg_reserve_data['visit_datetime'].dt.date

hpg_reserve_data['reserve_datetime'] = pd.to_datetime(hpg_reserve_data['reserve_datetime'])
hpg_reserve_data['reserve_hour'] = hpg_reserve_data['reserve_datetime'].dt.hour
hpg_reserve_data['reserve_date'] = hpg_reserve_data['reserve_datetime'].dt.date

#calculate reservation time difference 
hpg_reserve_data['res_vis_diff'] = (hpg_reserve_data['visit_datetime']-hpg_reserve_data['reserve_datetime']).apply(
    lambda x : x.total_seconds()/3600)

hpg_reserve_data.rename(columns={'reserve_visitors':'hpg_reserve_visitors'},inplace=True)
hpg_reserve_data.to_csv('hpg_reserve_data.csv',index=False)



In [ ]:
hpg_reserve_data.head()

*   The hpg_reserve data is merged with store_relation_info to get the corresponding air_store_id into **hpg_reserve_data** dataframe.
*   It is then merged with the hpg_store_info to get the area, genre, latitude and longitude for each hpg_store_id.   
*   The visit_datetime column has been changed into datetime from object dtype.

In [ ]:
total_air_reserve = air_reserve_data.groupby(['air_store_id','visit_date'],as_index=False)['air_reserve_visitors'].sum()
total_hpg_reserve =  hpg_reserve_data.groupby(['air_store_id','visit_date'],as_index=False)['hpg_reserve_visitors'].sum()
air_data = pd.merge(air_data,total_air_reserve,on=['air_store_id','visit_date'],how='left')
air_data = pd.merge(air_data,total_hpg_reserve,on=['air_store_id','visit_date'],how='left')
air_data.fillna(value=0,inplace=True)

In [ ]:
air_data.head()

*  The number of reserve visitors for each restaurant per day is kept in the total_air_reserve and total_hpg_reserve dataframes.
*  The air and hpg reserve visitor information is taken from their individual reserve dataset and merged with the air_data for comparing with the number of visitors. 

### 6. Individual Data Visualization

Now, we will try to analyze the different features for every individual datasets to get a better understanding of the data provided to us.

### 6.1. Train data

#### 6.1.1. Number of visitors each day in the training dataset

In [ ]:
plt1 = air_data.groupby(['visit_date'], as_index=False).agg({'visitors': np.sum})
plt1=plt1.set_index('visit_date')
plt1.plot(figsize=(15, 6))
plt.ylabel("All Visitors")
plt.title("Visitor each day")
plt.show()

**Observations:-**
*         The above plot depits the fact that there is almost 150% hike in the number of restaurants during mid of 2016.
*         The reason behind the hike might be caused by including new restaurants to the AIR database in mid 2016.
*         The ups and downs that we see might be due to the weekday and weekends.
*         The sharp decrese on 1st of Jan is due to the new-year's eve, as most of the restaurants stay closed on new-years day.

#### 6.1.2. Average number of visitors per restaurant per day

In [ ]:
plt2 = air_data.groupby(['air_store_id'])['visitors'].mean().to_frame()
f,ax = plt.subplots(1,1, figsize=(10,6))
plt.hist(x=plt2.visitors.values, bins =100)
plt.xlabel('Visitors')
plt.ylabel('Count')
plt.title('Average visitors per restaurant per day')
plt.show()



**Observations:-**

*        The number of average visitors per restaurant per day almost follows a normal curve with mean visitors approx. 20, having a slight right skewness.
*        There are a large number of restaurants which have capacity less than 20. It explains the fact that there is a large number of small restaurants in Japan.
*        There are very few luxurious restaurants that are visited by large number of people (>120).

The following boxplot gives a better analysis of the same data to understand the ranges and outliers in air_data.

In [ ]:
temp = air_data.groupby(['air_store_id'])['visitors'].mean().to_frame()
f,ax = plt.subplots(1,1, figsize=(10,6))
sns.boxplot(y='visitors', data=temp,ax=ax)
plt.title('Boxplot of average visitors per restaurant per day')
plt.show()

**Observations:-**
*   The minimum number of visitors that we can observe from this plot is almost reaching zero.
*   The mean of the visitors is 20(approx).
*   The maximum number of visitors is between 55-60.
*   We observe certain very high values (outlier) greater than 60 and and even greater than 100 visitors.
*   25th percentile and 75th percentile values are 13(approx) and 30(approx) respectively.



#### 6.1.3. PDF of average reserve visitors per restaurant

In [ ]:
temp_1 = total_air_reserve.groupby(['air_store_id'],as_index=False)['air_reserve_visitors'].mean()
temp_2 = total_hpg_reserve.groupby(['air_store_id'],as_index=False)['hpg_reserve_visitors'].mean()

f,ax = plt.subplots(1,1, figsize=(10,6))
sns.distplot(a=temp_1.air_reserve_visitors.values, ax=ax, label='air_reserve_visitors')
sns.distplot(a=temp_2.hpg_reserve_visitors.values, ax=ax, label='hpg_reserve_visitors')
plt.xlabel('Visitors Reserved')
plt.ylabel('Density')
plt.legend()
plt.title('PDF of average reserve visitors per restaurant')
plt.show()

**Observations:-**

*    The spread of AIR reservations is higher than that of HPG reservations.
*    There is a large number of reservations in HPG with visitors count between 5 to 10.
*    There are few reservations in HPG where the visitors count is more than 20 or even reaching 40.
*    Even in AIR, the maximum number of visitors registered is 40, but the number of registrations are more than that of HPG.
*    The number of unregistered visitors is far more than the number of registered visitors.


### 6.2. Air Reservation Data

#### 6.2.1 Number of reserve visitors per day

In [ ]:
airR1 = air_reserve_data.groupby(['visit_date'], as_index=False).agg({'air_reserve_visitors': np.sum})
airR1=airR1.set_index('visit_date')
airR1.plot(figsize=(15, 6))
plt.ylabel("Number of Reserve Visitors")
plt.title("Reserve Visitors each day")
plt.show()

**Observations:-**
*   There were much fewer reservations made in 2016 through the air system. 
*   The volume only increased during the end of that year during the month of festivities. 
*   In 2017 the visitor numbers stayed strong. 
*   We can see a flat line after July 2016 in the number of reserve visitors which shows that there is no data between August to October, 2016.
*   There is a drastic fall in number of reserve visitors on 1st January as most of the restaurants stay closed on new-years day.


#### 6.2.2. Visit Hour and Reservation-Visit Hour Gap analysis for air_reserve

In [ ]:
airR2 = air_reserve_data.groupby(['visit_hour'], as_index=False).agg({'air_reserve_visitors': np.sum})
airR3 = air_reserve_data.groupby(['res_vis_diff'], as_index=False).agg({'air_reserve_visitors': np.sum})

#plot
fig = plt.figure(figsize=(20, 6)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1.5, 2.5]) 
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])
sns.barplot(x="visit_hour",y="air_reserve_visitors",data=airR2,ax=ax0)
sns.barplot(x="res_vis_diff",y="air_reserve_visitors",data=airR3[(airR3['res_vis_diff'] <= 80)],ax=ax1)
ax0.set_xlabel("Visit Hour")
ax0.set_ylabel("Number of Reserve Visitor")
ax1.set_xlabel('Reservation-Visit Hour Gap')
ax1.set_ylabel("Number of Reserve Visitor")

for ax in [ax0,ax1]:
    for label in ax.get_xticklabels():
        label.set_rotation(90) 
        
plt.tight_layout(pad = 3.0)        
plt.show()

**Observations on Visit Hour:-**
*   Most people make reservation for dinner (6pm to 8pm).
*   There are very few reservations in the morning.

**Observations on Reservation-Visit Hour Gap:-**
*   Making reservation just a few hours before visiting the restaurant is the most common practice.
*   The plot shows that there is a nice 24-hour gap pattern between reservation and visit.
*   Very long time gaps between reservation and visit are not uncommon.

#### 6.2.3. Number of restaurants per area covered by AIR_RESERVE dataset

In [ ]:
# Number of restaurent in area: Air Data
airS1 = air_reserve_data['air_area_name'].value_counts().reset_index().sort_index()
airS2 = air_reserve_data['air_genre_name'].value_counts().reset_index().sort_index()
fig,ax = plt.subplots(1,2)
sns.barplot(y='index' ,x='air_area_name',data=airS1.iloc[:15],ax=ax[0])
sns.barplot(y='index' ,x='air_genre_name',data=airS2.iloc[:15],ax=ax[1])
fig.set_size_inches(20,7, forward=True)
ax[0].set_ylabel('Number of Restaurent')
ax[1].set_ylabel('Number of Restaurent')
plt.tight_layout(pad=3.0)
plt.show()

**Observations on area:-**
*    The restaurants in Japan are spreaded over 71 areas out of which the top 15 areas are shown above.
*    Tōkyō-to Shibuya-ku Shibuya has the largest number of restaurants.

**Observations on genre:-**
*    The restaurants in Japan is subdivided in 14 food genres.
*    Findings from the air_reserve dataset shows that Izakaya is the most popular genre in Japan. The second most popular genre in Japan is Italian/French.
*    International cuisine,Asian and Karaoke/Party are the least preferred genre.

To start a restaurant business in Japan, choosing restaurant area and food genre will be an important decision.

### 6.3. HPG Reservation Data
#### 6.3.1. Number of reserve visitors per data

In [ ]:
hpgR1 = hpg_reserve_data.groupby(['visit_date'], as_index=False).agg({'hpg_reserve_visitors': np.sum})
hpgR1=hpgR1.set_index('visit_date')
hpgR1.plot(figsize=(15, 6))
plt.ylabel("Sum of Visitors")
plt.title("Visitor each day")
plt.show()

**Observations:-**
*   The hpg reservation follow a more orderly pattern.
*   There is a clear spike in Dec 2016 which might caused due to festivities at that time of year.

#### 6.3.2. Visit Hour and Reservation-Visit Hour Gap analysis for air_reserve

In [ ]:
hpgR2 = hpg_reserve_data.groupby(['visit_hour'], as_index=False).agg({'hpg_reserve_visitors': np.sum})
hpgR3 = hpg_reserve_data.groupby(['res_vis_diff'], as_index=False).agg({'hpg_reserve_visitors': np.sum})

#plot
fig = plt.figure(figsize=(25, 6)) 
gs = gridspec.GridSpec(1, 2, width_ratios=[1.5, 2.5]) 
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])
sns.barplot(x="visit_hour",y="hpg_reserve_visitors",data=hpgR2,ax=ax0)
sns.barplot(x="res_vis_diff",y="hpg_reserve_visitors",data=hpgR3[(hpgR3['res_vis_diff'] <= 80)],ax=ax1)
ax0.set_xlabel('Visit Hour')
ax0.set_ylabel('Number of Reserve Visitor')
ax1.set_xlabel('Reservation-Visitor Hour Gap')
ax1.set_ylabel('Number of Reserve Visitor')
for ax in [ax0,ax1]:
    for label in ax.get_xticklabels():
        label.set_rotation(90) 

**Observations on Visit Hour:-**
*   The hpg_reserve data also shows that most people make reservation for dinner (6pm to 8pm).
*   There are very few reservations in the morning.

**Observations on Reservation-Visit Hour Gap:-**
*   Making reservation just a few hours before visiting the restaurant is comparable to the 24-hour pattern, whereas in air_reserve the number of reservation in the last moment was more.
*   Very long time gaps between reservation and visit are not uncommon.

#### 6.3.3. Number of restaurants per area covered by HPG_RESERVE dataset

In [ ]:
# Number of restaurent in area: HPG store
hpgS1=hpg_reserve_data['hpg_area_name'].value_counts().reset_index().sort_index()
hpgS2=hpg_reserve_data['hpg_genre_name'].value_counts().reset_index().sort_index()
fig,ax = plt.subplots(1,2)
sns.barplot(y='index' ,x='hpg_area_name',data=hpgS1.iloc[:15],ax=ax[0])
sns.barplot(y='index' ,x='hpg_genre_name',data=hpgS2.iloc[:15],ax=ax[1])
fig.set_size_inches(20,7, forward=True)
ax[0].set_ylabel('Number of Restaurent')
ax[1].set_ylabel('Number of Restaurent')
plt.tight_layout(pad=3.0)
plt.show()


**Observations on area:-**
*    The hpg_restaurants in Japan are spreaded over 33 areas out of which the top 15 areas are shown above.
*    Hokkaidō Asahikawa-shi 3 Jōdōri has the largest number of restaurants.

**Observations on genre:-**
*    The restaurants in Japan is subdivided in 14 food genres.
*    Findings from the hpg_reserve dataset shows that Japanese style is the most popular genre. The second most popular genre for hpg restaurants is International Cuisine.

### 6.4. Number of Restaurants vs Visit Date in Train Data

In [ ]:
f,ax = plt.subplots(1,1, figsize=(15,6))
stores= air_data.groupby(['visit_date'])['air_store_id'].size()
stores.plot(kind='line',  color= 'chocolate', grid=True, ax=ax, legend=True)
plt.ylabel("Number of Unique Resturant")
plt.xlabel("Visit Date")
plt.title("Number of Restaurants vs Visit Date in Train Data")
plt.show()

**Observations:-**

*        The above plot depicts the fact that there is almost 150% hike in the number of restaurants during mid of 2016.
*        The reason behind the hike is that there is an addition of 500(approx) new restaurants to the AIR database in mid 2016.
*        The ups and downs that we see is may be due to the weekday and weekends.
*        The sharp decrese on 1st of Jan is due to the new-year's eve, as most of the restaurants stay closed on new-years day.
*        In total we have records of almost 800 japanese restaurants.

### 6.5. Comparision between number of visitors and number of reservations

In [ ]:
f,ax = plt.subplots(1,1,figsize=(20,8))
plt1 = air_data.groupby(['visit_date'])['visitors'].sum().to_frame()
plt2 = air_reserve_data.groupby(['visit_date'])['air_reserve_visitors'].sum().to_frame()
plt3 = hpg_reserve_data.groupby(['visit_date'])['hpg_reserve_visitors'].sum().to_frame()
plt1.plot(color='salmon', kind='line', ax=ax)
plt2.plot(color='cornflowerblue', kind='line', ax=ax)
plt3.plot(color='y', kind='line', ax=ax)
plt.legend()
plt.ylabel("Sum of Visitors")
plt.xlabel("Visit Date")
plt.title("Visitor and Reservations")
plt.show()

**Observations:-**

*        The abrupt hike in the middle of the 2016 is most probably due to the addition of new restaurants(as explained in previous plot).
*        It can be easily observed that the number of non-registered visitors is far more than the number of registered visitors.
*        There is a sharp decline at new years eve as most of the restaurants remain close on new year eve.
*        The number of registration in AIR is more than that of HPG.
*        The maximum numbers of visitors is observed in the month of December, since there are a number of festivals in December.



### 6.6. Genre-wise comparison 
#### 6.6.1. Genre-wise comparison for visitors

In [ ]:
f,ax = plt.subplots(1,1,figsize=(15,6))
plt1 = air_data.groupby('air_genre_name')['visitors'].sum().reset_index().nlargest(14, 'visitors')
sns.barplot(x='visitors', y='air_genre_name', data=plt1)
plt.show()

**Observations:-**
*   Izakaya has the highest number of visitors and hence is the most popular genre.
*   Cafe/Sweets is the secondmost popular popular genre.

#### 6.6.2 Genre-wise comparison for hpg_reserve_visitors and air_reserve_visitors

In [ ]:
f,ax = plt.subplots(1,1, figsize=(15,6))
genre = air_data.groupby(['air_genre_name'])['air_reserve_visitors','hpg_reserve_visitors'].sum()
genre.sort_values(by=['air_reserve_visitors','hpg_reserve_visitors'],inplace=True)
genre.plot(kind='barh',color= ['salmon','cornflowerblue'], grid=True, ax=ax, legend=True)
plt.ylabel('Genres')
plt.xlabel('Reserve Visitors Count')
plt.legend(loc='center right')
plt.title("Reserve Visitors by Genre", loc='center')
plt.show()

**Observation:-**
*     Izakaya is the most popular genre. There are more reservations made using air service than that of hpg for Izakaya.
*     Here Italian/French is the second most popular genre.
*     Bar/Cocktail have comparable air and hpg reserve visitors.
*     Aisan and International Cuisine are the least popular.
*     Surprisingly the Japanese Food is the 4th popular genre in Japan.



### 6.7. Average visitors on Holidays & Non-Holidays

In [ ]:
temp = air_data[['holiday_flg','visitors']].groupby(['holiday_flg'])['visitors'].mean()
temp.plot(kind='bar',color= ['green','blue'],figsize=(10,6))
plt.ylabel('Average Visitors')
plt.xlabel('Non-holiday=0 & Holiday=1')
plt.title('Average visitors on Holidays & Non-Holidays')
plt.show()

**Observation:-**

*        It is observed from the plot and it is obvious to have more visitors on holidays than working days.
*        Even then the difference between the visitors on holidays and working days is not significantly large, which is due to weekend effect.
*        It is also observed from the above plot that on most of the holidays, restaurants are open.
*        While processing data, we must take into account the holidays that come on weekends, such holidays should only be considered as weekends not as holidays just to take the weekends effect into account.



### 6.8. Area-wise Analysis of Train data

In [ ]:
import folium
from folium import plugins

location =air_data.groupby(['air_store_id','air_genre_name'])['latitude','longitude'].mean().reset_index()
locationlist = location[['latitude', 'longitude']]
locationlist = locationlist.values.tolist()
map2 = folium.Map(location=[39, 139], 
                        tiles = "Stamen Toner", width=1000, height=500,
                        zoom_start = 5)
marker_cluster=plugins.MarkerCluster().add_to(map2)
for point in range(0, len(location)):
    folium.Marker(locationlist[point], popup=location['air_genre_name'][point], 
    icon=folium.Icon(color='white', icon_color='red', 
                     #icon='fa fa-info-circle',
                     icon='fa fa-circle-o-notch fa-spin',
                     angle=0, 
                     prefix='fa')).add_to(marker_cluster)
map2

**Observation:-**
*   This map shows the distribution of restaurants in Japan.
*   The restaurants are located in 7 major areas. On zooming on each of them shows the distribution of restaurants in each of these location.
*   Tokyo has the largest number of restaurants.

### 6.9. Distribution of visitors in each genre

In [ ]:
ax = sns.FacetGrid(air_data, col="air_genre_name", col_wrap=4, height=5, hue='air_genre_name',margin_titles=True,
                  aspect=1.5, palette='husl', ylim=(0,150))
ax = ax.map(sns.lineplot, "visit_date", "visitors",  marker=".", linewidth = 0.5)
plt.show()

**Observations:-**
*  People visit the genres Izakaya, Cafe/Sweets, Italian/French and Dining bar consistently throughout the year and hence these genres have highest number of visitors.
* Asian, International cuisine and Karaoke/Party are the new genres of restaurants that started in mid 2016.


### 6.10. Visitors Distribution of April, 2016

We have to predict the number of visitors for second and third week of April, 2017. So we are analyzing the number of visitors in the month of April,2016 as that may give rise to some monthly trend.

In [ ]:
start_date = pd.to_datetime('2016-04-01')
end_date = pd.to_datetime('2016-04-30')
confi_plot=air_data.loc[(air_data['visit_date'] > start_date) & (air_data['visit_date'] <= end_date), air_data.columns] 

confidenceIntervals = [90, 50, 10]

ax = plt.subplots(1,1, figsize=(20,6))

colorPalette = sns.color_palette("ch:2.5,-.2,dark=.7", n_colors=len(confidenceIntervals))[::-1]

for jj, ii in enumerate(confidenceIntervals):
    ax = sns.lineplot(x="visit_date", y="visitors", data=confi_plot, ci=ii, label=str(ii), 
                      color=colorPalette[jj])

ax.legend(fancybox=True, framealpha=1, shadow=True, borderpad=1)
plt.show()

**Observations:-**
*   Confidence Interval refers to the probability that a population parameter will fall between a set of values for a certain proportion of time. Confidence intervals measure the degree of uncertainty or certainty in a sampling method. They can take any number of probability limits, with the most common being a 95% or 99% confidence level.
*   Here we are showing the confidence in number of visitors on the month of april 2016 with the confidence level 10%,50% and 90%
*   The points lying on the midddle line have a higher confidence level (90%). As the colour fades the confidence level decreases.

## 7. Feature Engineering

### 7.1. New Feature: Reserve_datetime_difference


1. The air_store_id is mapped onto the corresponding hpg_store_id data from the store_id_relation.
2. The reserve_datetime_difference column containing the difference between the visit_datetime and reserve_datetime is added.
3. Derived columns: 
   - rdd_mean(mean of reserve_datetime_diff), 
   - rv_sum(total reserve visitor), 
   - rv_mean(mean of reserve visitors) are added by grouping the data on columns air_store_id and visit_datetime.


In [ ]:
#getting the air_store_id of corresponding hpg_stores from store_id_relation
hpg_reserve = pd.merge(hpg_reserve,store_id_relation,on=['hpg_store_id'],how='inner')

In [ ]:
#hpg_reserve preprocessing
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['visit_datetime'] = hpg_reserve['visit_datetime'].dt.date
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])
hpg_reserve['reserve_datetime'] = hpg_reserve['reserve_datetime'].dt.date
hpg_reserve['reserve_datetime_diff'] = hpg_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)

tmp1 = hpg_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_visitors':'rv_sum'})
tmp2 = hpg_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rdd_mean', 'reserve_visitors':'rv_mean'})
hpg_reserve= pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [ ]:
#air_reserve preprocessing
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['visit_datetime'] = air_reserve['visit_datetime'].dt.date
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['reserve_datetime'] = air_reserve['reserve_datetime'].dt.date
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)

tmp1 = air_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_visitors':'rv_sum'})
tmp2 = air_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rdd_mean', 'reserve_visitors':'rv_mean'})
air_reserve= pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

### 7.2. Extracting dow(day of week),  year, month from visit_date 

- In data visualization, we found out that number of visitors shows some trend depending on the day of week and month of year.
- So, we are adding new columns dow, month and year to the train and test dataframe.

In [ ]:
train['visit_date'] = pd.to_datetime(train['visit_date'])
train['dow'] = train['visit_date'].dt.dayofweek
train['year'] = train['visit_date'].dt.year
train['month'] = train['visit_date'].dt.month
train['month_name'] = train['visit_date'].dt.month_name()
train['visit_date'] = train['visit_date'].dt.date

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

In [ ]:
test['visit_date'] = test['id'].map(lambda x: str(x).split('_')[2])
test['air_store_id'] = test['id'].map(lambda x: '_'.join(x.split('_')[:2]))
test['visit_date'] = pd.to_datetime(test['visit_date'])
test['dow'] = test['visit_date'].dt.dayofweek
test['year'] = test['visit_date'].dt.year
test['month'] = test['visit_date'].dt.month
test['month_name'] = test['visit_date'].dt.month_name()
test['visit_date'] = test['visit_date'].dt.date

#### 7.2.2. Adding new feature non_working in date_info dataframe

- As previously observed in data visualization, mean number of visitors in weekends is comparable to that in holidays. 
- A new feature, non_working is added, where along with holidays,  Saturday and Sunday are marked 1(non-working).
- Significant difference in mean number of visitors between working and non-working days helps future prediction of number of visitors.

In [ ]:
date_info['non_working'] = np.where(date_info['day_of_week'].isin(['Saturday', 'Sunday']) | date_info['holiday_flg'] == 1, 1, 0)
date_info = date_info.drop('holiday_flg', axis = 1)

#label encoding day_of_week which is a categorical data
lbl = preprocessing.LabelEncoder()
date_info['day_of_week'] = lbl.fit_transform(date_info['day_of_week'])

date_info = date_info.rename(columns={'calendar_date':'visit_date'})
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
date_info['visit_date'] = date_info['visit_date'].dt.date

train_set = pd.merge(train, date_info, how='left', on=['visit_date'])
test_set = pd.merge(test, date_info, how='left', on=['visit_date']) 

In [ ]:
fig, ax =plt.subplots(1,2)
fig.set_size_inches(15,4, forward=True)

sns.boxplot(x = "day_of_week", y = "visitors", data=train_set,ax=ax[0])
sns.boxplot(x = "month_name",y = "visitors", data=train_set,ax=ax[1])
ax[0].set_xlabel('Day of week')
ax[0].set_ylabel('Median Visitors')
ax[1].set_xlabel('Month of Year')
ax[1].set_ylabel('Median Visitors')
for ax in ax:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:
fig, ax =plt.subplots(1,2)
fig.set_size_inches(15,4, forward=True)

month_day_log = train_set.copy()
month_day_log['visitors'] =  np.log1p(month_day_log['visitors'])

sns.boxplot(x = "day_of_week", y = "visitors", data=month_day_log,ax=ax[0])
sns.boxplot(x = "month_name",y = "visitors", data=month_day_log,ax=ax[1])
ax[0].set_xlabel('Day of week')
ax[0].set_ylabel('Median Visitors')
ax[1].set_xlabel('Month of Year')
ax[1].set_ylabel('Median Visitors')
for ax in ax:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

#### Effect of non_working feature
- Significant difference in mean number of visitors between working and non-working days helps future prediction of number of visitors.


In [ ]:
temp = train_set[['non_working','visitors']].groupby(['non_working'])['visitors'].mean()
temp.plot(kind='bar',color= ['green','blue'],figsize=(10,6))
plt.ylabel('Average Visitors')
plt.xlabel('Working=0 & Non-working=1')
plt.title('Average visitors on Working & Non-Working')
plt.show()

In [ ]:
#merging hpg_reserve data with train and test set
train_set = pd.merge(train_set, hpg_reserve, how='left', on=['air_store_id','visit_date']) 
test_set = pd.merge(test_set, hpg_reserve, how='left', on=['air_store_id','visit_date'])

#merging air_reserve data with train and test set
train_set = pd.merge(train_set, air_reserve, how='left', on=['air_store_id','visit_date']) 
test_set = pd.merge(test_set, air_reserve, how='left', on=['air_store_id','visit_date'])

In [ ]:
#label encoding air_store_id in train and test dataset as it is categorical value
lbl = preprocessing.LabelEncoder()
train_set['air_store_id2'] = lbl.fit_transform(train_set['air_store_id'])
test_set['air_store_id2'] = lbl.transform(test_set['air_store_id'])

In [ ]:
train_set = train_set.fillna(0)
test_set = test_set.fillna(0)

In [ ]:
#merging both the hpg_reserve and air_reserve dataset to train and test set introduces a pair of rv_sum, rv_mean and rdd_mean
#so we add up total reserve visitors (rv_sum) 
#find the mean of pair of rv_mean
#find the mean of pair of rdd_mean

train_set['rv_sum'] = train_set['rv_sum_x'] + train_set['rv_sum_y']
train_set['rv_mean'] = (train_set['rv_mean_x'] + train_set['rv_mean_y'])/2
train_set['rdd_mean'] = (train_set['rdd_mean_x'] + train_set['rdd_mean_y'])/2

test_set['rv_sum'] = test_set['rv_sum_x'] + test_set['rv_sum_y']
test_set['rv_mean'] = (test_set['rv_mean_x'] + test_set['rv_mean_y'])/2
test_set['rdd_mean'] = (test_set['rdd_mean_x'] + test_set['rdd_mean_y'])/2

In [ ]:
#remove the extra columns after updating the sum and mean in new columns
train_set.drop(columns = {'rv_sum_x', 'rdd_mean_x', 'rv_mean_x', 'rv_sum_y', 'rdd_mean_y', 'rv_mean_y'}, inplace  = True)
test_set.drop(columns = {'rv_sum_x', 'rdd_mean_x', 'rv_mean_x', 'rv_sum_y', 'rdd_mean_y', 'rv_mean_y'}, inplace  = True)

### 7.2. Genre and Area wise restaurants grouping

   We are creating a new feature that counts the number of restaurants of a particular genre in a particular area.


In [ ]:
#merging store information with train and test set and label encoding the air_genre_name and air_area_name, which are categorical data
lbl = preprocessing.LabelEncoder()
train_set = pd.merge(train_set, air_store_info, on=['air_store_id'], how = 'left')
test_set = pd.merge(test_set, air_store_info, on=['air_store_id'], how = 'left')

train_set['air_genre_name'] = lbl.fit_transform(train_set['air_genre_name'])
train_set['air_area_name'] = lbl.fit_transform(train_set['air_area_name'])

test_set['air_genre_name'] = lbl.fit_transform(test_set['air_genre_name'])
test_set['air_area_name'] = lbl.fit_transform(test_set['air_area_name'])

### 7.3. Working with Latitude and Longitude columns
   **(Included in Final Improvements)**         

In [ ]:
train_set['var_max_lat'] = train_set['latitude'].max() - train_set['latitude']
train_set['var_max_long'] = train_set['longitude'].max() - train_set['longitude']
test_set['var_max_lat'] = test_set['latitude'].max() - test_set['latitude']
test_set['var_max_long'] = test_set['longitude'].max() - test_set['longitude']

train_set['lon_plus_lat'] = train_set['longitude'] + train_set['latitude'] 
test_set['lon_plus_lat'] = test_set['longitude'] + test_set['latitude']

In [ ]:
col = [c for c in train_set if c not in ['id', 'air_store_id', 'visit_date','visitors', 'month_name']]

### 8. MODEL BUILDING
- We try out different models

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def RMSLE(y, pred):
    y = np.expm1(y)
    pred = np.expm1(pred)
    return np.sqrt(np.square(np.log(pred + 1) - np.log(y + 1)).mean())

model1 = LinearRegression()
model2 = DecisionTreeRegressor()
model3 = RandomForestRegressor()     # New model used ( Included in the final improvements ) 
model4 = GradientBoostingRegressor()
model5 = XGBRegressor()
model6 = LGBMRegressor()    # New model used ( Included in the final improvements ) 

In [ ]:
param_decision={
    "max_depth" : [1,2,3,4,5,6,None],
    "min_samples_split" : [12,15,17,20],
    "max_leaf_nodes" : [20,30,40],
}

In [ ]:
grid = GridSearchCV(model2,param_grid = param_decision,n_jobs = -1)
grid.fit(train_set[col],train_set['visitors'])

In [ ]:
grid.best_estimator_

In [ ]:
param_reg={
    "max_depth" : [8,10,12],
    #"learning_rate" : [0.1,0.2,0.3]
}

In [ ]:
grid = GridSearchCV(model3,param_grid = param_reg,n_jobs = -1)
grid.fit(train_set[col],train_set['visitors'])


In [ ]:
grid.best_estimator_

In [ ]:
grid = GridSearchCV(model4,param_grid = param_reg,n_jobs = -1)
grid.fit(train_set[col],train_set['visitors'])

In [ ]:
grid = GridSearchCV(model5,param_grid = param_reg,n_jobs = -1)
grid.fit(train_set[col],train_set['visitors'])

In [ ]:
grid = GridSearchCV(model6,param_grid = param_reg,n_jobs = -1)
grid.fit(train_set[col],train_set['visitors'])

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_estimator_

In [ ]:
model1 = LinearRegression()
model2 = DecisionTreeRegressor(max_depth=3, max_leaf_nodes=20, min_samples_split=12)
model3 = RandomForestRegressor(max_depth=8)
model4 = GradientBoostingRegressor(max_depth=8)
model5 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', 
             learning_rate=0.300000012, max_delta_step=0, max_depth=8,
             min_child_weight=1,
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
model6 = LGBMRegressor(max_depth=8,objective='regression', num_leaves=60, learning_rate=0.01, n_estimators=10000)

model1.fit(train_set[col], np.log1p(train_set['visitors'].values))
model2.fit(train_set[col], np.log1p(train_set['visitors'].values))
model3.fit(train_set[col], np.log1p(train_set['visitors'].values))
model4.fit(train_set[col], np.log1p(train_set['visitors'].values))
model5.fit(train_set[col], np.log1p(train_set['visitors'].values))
model6.fit(train_set[col], np.log1p(train_set['visitors'].values))


preds1 = model1.predict(train_set[col])
preds2 = model2.predict(train_set[col])
preds3 = model3.predict(train_set[col])    # New model used ( Included in the final improvements )
preds4 = model4.predict(train_set[col])
preds5 = model5.predict(train_set[col])
preds6 = model6.predict(train_set[col])    # New model used ( Included in the final improvements )
preds56 =  (0.5*preds5) + (0.5*preds6)

print('RMSLE LinearRegressor: ', RMSLE(np.log1p(train_set['visitors'].values), preds1))
print('RMSLE DecisionTreeRegressor: ', RMSLE(np.log1p(train_set['visitors'].values), preds2))
print('RMSLE RadomForest: ', RMSLE(np.log1p(train_set['visitors'].values), preds3))
print('RMSLE GradientBoostingRegressor: ', RMSLE(np.log1p(train_set['visitors'].values), preds4))
print('RMSLE XGBRegressor: ', RMSLE(np.log1p(train_set['visitors'].values), preds5))
print('RMSLE LGBMRegressor: ', RMSLE(np.log1p(train_set['visitors'].values), preds6))
print('RMSLE Equally Weighted Ensemble: ', RMSLE(np.log1p(train_set['visitors'].values), preds56))


In [ ]:
# LINEAR REGRESSOR

# preds1 = model1.predict(test_set[col])

# test_set['visitors'] = preds1
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

In [ ]:
# DECISION TREE REGRESSOR

# preds2 = model2.predict(test_set[col])

# test_set['visitors'] = preds2
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

In [ ]:
# RANDOM FOREST

# preds3 = model3.predict(test_set[col])

# test_set['visitors'] = preds3
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

In [ ]:
# GRADIENT BOOSTING REGRESSOR

# preds2 = model2.predict(test_set[col])

# test_set['visitors'] = preds2
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

In [ ]:
# # XGBOOST REGRESSOR

# preds5 = model5.predict(test_set[col])

# test_set['visitors'] = preds5
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

In [ ]:
# # LIGHT GBM

# preds6 = model6.predict(test_set[col])

# test_set['visitors'] = preds6
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

In [ ]:
#Ensemble Model

# preds56 = 0.5 * model3.predict(test_set[col]) + 0.5 * model4.predict(test_set[col])

# test_set['visitors'] = preds5
# test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
# sub1 = test_set[['id','visitors']].copy()
# sub1.to_csv('submission.csv', index=False)

# Finding the proper ensemble ratios to get minimum RMSLE ERROR 
**( Included in the final improvements )**

In [ ]:
min_i = 0
min_rmsle = 1
for i in np.arange(0.0, 1.0, 0.001):
    pred_i = i*preds5 + (1-i)*preds6
    rmsle_i = RMSLE(np.log1p(train_set['visitors'].values), pred_i)
    if(rmsle_i < min_rmsle):
        min_rmsle = rmsle_i
        min_i = i
        
print('Min i = ', min_i)
print('Min rmsle = ', min_rmsle)

# Ensemble Model : LGBM and XGBoost 
**(Included in the final improvements)**

In [ ]:
#Ensemble Model : LGBM and XGBoost

pred_en = 0.317 * model5.predict(test_set[col]) + (1-0.317) * model6.predict(test_set[col])

test_set['visitors'] = pred_en
test_set['visitors'] = np.expm1(test_set['visitors']).clip(lower=0.)
sub1 = test_set[['id','visitors']].copy()
sub1.to_csv('submission.csv', index=False)

In [ ]:
submission = pd.read_csv("./submission.csv")
submission.head()